In [3]:
# import necessary libraries
import numpy as np
import plotly.graph_objects as go
# plotly will help us generate an interactive graph

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# ----------- USER SETTINGS ------------
faces = [1, 2, 3, 4, 5, 6]             # Die faces
probabilities = [1/6]*6                # Change to uneven, e.g. [0.05,0.1,0.1,0.2,0.25,0.3]
n_throws = 10                          # Number of throws per sample
n_samples = 1000                       # Number of samples (frames in animation)
frame_step = 10                        # How many samples per frame (for speed)
# -------------------------------------

We must now perform the following checks
- The probabilities sum to 1
- The count of probabilities = the number of faces

## Assert Statement

The assert statement in Python is a debugging aid that tests a condition and triggers an error if the condition is not true. The basic syntax is -


    assert condition, "Optional error message"

- If the condition is True, nothing happens and execution continues
- If the condition is False, Python raises an AssertionError

#### When we use Assert statements

- Debugging: Catch "should never happen" situations
- Development: Document assumptions in your code
- Testing: Quick sanity checks during development

#### Important Notes
Don't use for data validation in production code (users can disable assertions)
Assertions can be globally disabled with the -O (optimize) flag:

    python -O script.py  # All assert statements are ignored

## Python Error Handling: assert vs raise vs try-except

| Aspect | `assert` | `raise` | `try-except` |
|--------|----------|---------|--------------|
| **Purpose** | Debugging & sanity checks | Signal errors explicitly | Handle & recover from errors |
| **Production Use** | Can be disabled with `-O` flag | Always active | Always active |
| **Control Flow** | Stops execution if false | Stops execution unless caught | Continues execution after handling |
| **Syntax** | `assert condition, "message"` | `raise ExceptionType("message")` | `try: ... except: ...` |
| **Best For** | "Impossible" conditions, internal consistency | Expected errors, input validation, business rules | Error recovery, resource cleanup, graceful degradation |
| **Exception Type** | Always `AssertionError` | Any exception type | Catches any exception type |
| **Performance** | Removed in optimized mode | Always executed | Always executed |
| **Example Use Case** | Checking function preconditions | Validating user input | Handling file I/O errors |


In [ ]:
assert np.isclose(sum(probabilities), 1), "Probabilities must sum to 1."
assert len(faces) == len(probabilities), "Number of faces must equal number of probabilities."

We can now calculate theoretical values to show on the graph too

### Single Throw
**Expectation:**
$$E[X] = \sum_{i} p_i \cdot x_i$$

**Variance:**
$$\text{Var}(X) = \sum_{i} p_i \cdot (x_i - E[X])^2$$

### Sum of $n$ Independent Throws
All these are independent events, hence why we simply multiply with n the Expectation and Variance

**Expectation:**
$$E[S_n] = n \cdot E[X]$$

**Variance:**
$$\text{Var}(S_n) = n \cdot \text{Var}(X)$$

**Standard Deviation:**
$$\sigma_{S_n} = \sqrt{\text{Var}(S_n)} = \sqrt{n} \cdot \sigma_X$$

Where:
- $X$ is the outcome of a single throw
- $S_n$ is the sum of $n$ independent throws
- $x_i$ are the face values
- $p_i$ are their probabilities
- $n$ is the number of throws

In [ ]:
# Theoretical values
expectation_single = np.dot(faces, probabilities)
variance_single = np.dot(probabilities, (np.array(faces) - expectation_single)**2)
expectation_sum = n_throws * expectation_single
variance_sum = n_throws * variance_single
std_sum = np.sqrt(variance_sum)

In [ ]:
# Generate all sample sums
sample_sums = [np.sum(np.random.choice(faces, size=n_throws, p=probabilities)) for i in range(n_samples)]

## Create frames for animation

1. Setting up 'buckets' for our histogram:
Think of this as creating the empty boxes we'll put our results into. If we're throwing dice with faces 1-6, and we throw 10 dice, the total can range from 10 to 60. This creates boxes for every possible total score

2. The Animation Loop
We're going to create multiple pictures (frames) of our histogram. Each frame shows the histogram after adding more samples. frame_step = how many samples we add between each picture. Example: if frame_step=100, we take pictures after 100, 200, 300 samples, etc.
3. Counting Results for each frame
4.

In [ ]:
frames = []
bins = np.arange(n_throws * min(faces), n_throws * max(faces) + 2)

In [ ]:
for i in range(frame_step, n_samples + frame_step, frame_step):
    hist, edges = np.histogram(sample_sums[:i], bins=bins, density=False)
    centers = (edges[:-1] + edges[1:]) / 2
    frames.append(go.Frame(
        data=[go.Bar(x=centers, y=hist, marker_color='skyblue', opacity=0.7)],
        name=f"{i}"
    ))